In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.optim as optim

file_path = "Housing.csv"
df = pd.read_csv(file_path)

inputs = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
X = df[inputs].values
Y = df['price'].values
m = len(Y)

X_0 = np.ones((m, 1))
X = np.hstack((X_0, X))
theta = np.zeros(X.shape[1])

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
scaler = MinMaxScaler()
X_train[:, 1:] = scaler.fit_transform(X_train[:, 1:])
X_test[:, 1:] = scaler.transform(X_test[:, 1:])

In [35]:
# Training data
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32).view(-1,1)  # shape (N,1)

# Validation data
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32).view(-1,1)

In [36]:
params = torch.zeros((6,1), requires_grad=True)

def linear_model(X, params):
    B = params[0]
    W1 = params[1]
    W2 = params[2]
    W3 = params[3]
    W4 = params[4]
    W5 = params[5]

    U = B + W1*X[:,1:2] + W2*X[:,2:3] + W3*X[:,3:4] + W4*X[:,4:5] + W5*X[:,5:6]
    return U

def loss_fn(y_pred, y_true):
    return ((y_pred - y_true)**2).mean()

In [37]:
def training_loop(n_epochs, optimizer, params, X, y):
    for epoch in range(1, n_epochs+1):
        y_pred = linear_model(X, params)
        loss = loss_fn(y_pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 500 == 0:
            print("Epoch", epoch, "Loss=", loss.item())

    
    return params

In [38]:
learning_rate = 0.01
optimizer = optim.SGD([params], lr=learning_rate)
trained_params = training_loop(
    n_epochs=2000,
    optimizer=optimizer,
    params=params,
    X=X_train_tensor,
    y=Y_train_tensor
)

# Evaluate on validation set
with torch.no_grad():
    y_val_pred = linear_model(X_test_tensor, trained_params)
    val_loss = loss_fn(y_val_pred, Y_test_tensor)
    print("Validation loss:", val_loss.item())

# Show trained parameters
print("Trained parameters [B-W5] =", trained_params.detach().view(-1).numpy())

Epoch 500 Loss= 1699869229056.0
Epoch 1000 Loss= 1556255735808.0
Epoch 1500 Loss= 1481826238464.0
Epoch 2000 Loss= 1436944433152.0
Validation loss: 2440182366208.0
Trained parameters [B-W5] = [2508056.5 2933135.8 1395469.2 2351054.8 1603754.9 1423087.1]


In [39]:
learning_rates = [0.1, 0.01, 0.001, 0.0001]

for lr in learning_rates:
    print("Training with learning rate =", lr)
    params = torch.zeros((6,1), requires_grad=True)
    optimizer = optim.SGD([params], lr=lr)

    for epoch in range(1, 5001):
        y_pred = linear_model(X_train_tensor, params)
        loss = loss_fn(y_pred, Y_train_tensor)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            with torch.no_grad():
                y_val = linear_model(X_test_tensor, params)
                val_loss = loss_fn(y_val, Y_test_tensor).item()

            print("Epoch", epoch, "Training Loss=", loss.item(), "Validation Loss=", val_loss)

Training with learning rate = 0.1
Epoch 500 Training Loss= 1358686322688.0 Validation Loss= 2313595125760.0
Epoch 1000 Training Loss= 1350310166528.0 Validation Loss= 2293964472320.0
Epoch 1500 Training Loss= 1350024036352.0 Validation Loss= 2292591099904.0
Epoch 2000 Training Loss= 1350009225216.0 Validation Loss= 2292614692864.0
Epoch 2500 Training Loss= 1350008176640.0 Validation Loss= 2292681539584.0
Epoch 3000 Training Loss= 1350008176640.0 Validation Loss= 2292710899712.0
Epoch 3500 Training Loss= 1350008045568.0 Validation Loss= 2292719812608.0
Epoch 4000 Training Loss= 1350008045568.0 Validation Loss= 2292721909760.0
Epoch 4500 Training Loss= 1350008176640.0 Validation Loss= 2292722434048.0
Epoch 5000 Training Loss= 1350008176640.0 Validation Loss= 2292722434048.0
Training with learning rate = 0.01
Epoch 500 Training Loss= 1699869229056.0 Validation Loss= 2915957211136.0
Epoch 1000 Training Loss= 1556255735808.0 Validation Loss= 2649620742144.0
Epoch 1500 Training Loss= 1481826